<a href="https://colab.research.google.com/github/JayThibs/huggingface-course-cer-workshop/blob/main/notebooks/02_Fine_Tuning_with_Huggingface_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-Tuning Custom Model with Huggingface Transformers

Things I've added:

* More detailed explanations of concepts.
* Added dynamic padding for tokenization.
* Training with PyTorch Lightning to simplify training loop.
* Used custom dataset.

Note from Jacques: In order to speed up workshop prep, I'm going to be borrowing their example notebooks on fine-tuning models with the transformers library. Many of their example notebooks can be found here: https://huggingface.co/docs/transformers/notebooks. If you want to dive deeper into the library, you should look there.

# Installations

In [2]:
!pip install transformers==4.14.1 datasets==1.16.1 pytorch-lightning==1.5.6 -q

     |████████████████████████████████| 3.4 MB 6.8 MB/s 
     |████████████████████████████████| 298 kB 65.2 MB/s 
     |████████████████████████████████| 525 kB 89.0 MB/s 
     |████████████████████████████████| 3.3 MB 71.9 MB/s 
     |████████████████████████████████| 895 kB 56.5 MB/s 
     |████████████████████████████████| 61 kB 596 kB/s 
     |████████████████████████████████| 596 kB 70.7 MB/s 
     |████████████████████████████████| 1.1 MB 69.9 MB/s 
     |████████████████████████████████| 132 kB 78.5 MB/s 
     |████████████████████████████████| 243 kB 76.6 MB/s 
     |████████████████████████████████| 829 kB 70.4 MB/s 
     |████████████████████████████████| 332 kB 75.8 MB/s 
     |████████████████████████████████| 160 kB 84.2 MB/s 
     |████████████████████████████████| 192 kB 74.9 MB/s 
     |████████████████████████████████| 271 kB 57.7 MB/s 


# Imports

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from datasets import load_metric
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer

# Fine-tuning a pretrained model

In this tutorial, we will show you how to fine-tune a pretrained model from the Transformers library. In TensorFlow,
models can be directly trained using Keras and the `fit` method. In PyTorch, there is no generic training loop so
the 🤗 Transformers library provides an API with the class [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) to let you fine-tune or train
a model from scratch easily. Then we will show you how to alternatively write the whole training loop in PyTorch. 

**Jacques Note:** If you don't want to use the Huggingface Trainer API for some reason (like wanting more customization for your model), I would suggest using PyTorch Lightning with Huggingface since Lightning makes it so much easier to train models than Pure PyTorch.

Before we can fine-tune a model, we need a dataset. In this tutorial, we will show you how to fine-tune BERT on the
[IMDB dataset](https://www.imdb.com/interfaces/): the task is to classify whether movie reviews are positive or
negative. For examples of other tasks, refer to the [additional-resources](#additional-resources) section!

<a id='data-processing'></a>

## Preparing the datasets

In [4]:
#@title  { display-mode: "form" }
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/HyQgpJTkRdE" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

Instead of using a dataset from `datasets`, we'll be loading a custom model from Kaggle. We'll be loading a fake and real news dataset and train a model to detect whether a piece of text is "fake news" or "real (true) news". We could imagine adding supplementary data to create model that detects whether a statement a truthful or not, which is an important problem in AI Safety!

We'll start by downloading the data from my Google Drive.

## Downloading the Data

In [5]:
!gdown --id 1s_N6Bv48vSzdhx3Yw1v-3gtKHuv1gLVR

Downloading...
From: https://drive.google.com/uc?id=1s_N6Bv48vSzdhx3Yw1v-3gtKHuv1gLVR
To: /content/fake_real_news.zip
100% 42.4M/42.4M [00:00<00:00, 59.3MB/s]


In [6]:
!unzip fake_real_news.zip

Archive:  fake_real_news.zip
   creating: fake_real_news/
  inflating: __MACOSX/._fake_real_news  
  inflating: fake_real_news/Fake.csv  
  inflating: __MACOSX/fake_real_news/._Fake.csv  
  inflating: fake_real_news/True.csv  
  inflating: __MACOSX/fake_real_news/._True.csv  


## Data Exploration

Before we load our data with `load_dataset` from the `datasets` library, we're going to explore the data a bit and create train, validation, and test sets.

In [47]:
fake_df = pd.read_csv('fake_real_news/Fake.csv')
real_df = pd.read_csv('fake_real_news/True.csv')
fake_df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [48]:
fake_df.describe()

,title,text,subject,date
count,23481,23481,23481,23481
unique,17903,17455,6,1681
top,MEDIA IGNORES Time That Bill Clinton FIRED His...,,News,"May 10, 2017"
freq,6,626,9050,46


In [49]:
fake_df.subject.unique()

array(['News', 'politics', 'Government News', 'left-news', 'US_News',
       'Middle-east'], dtype=object)

In [50]:
fake_df.shape

(23481, 4)

Let's add the label to both dataframes before we concatenate them.

In [51]:
fake_df['label'] = 1 # Fake
real_df['label'] = 0 # Real

In [52]:
real_df.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0


Let's merge the datasets together so that we can create train, validation, test splits with both real and fake news.

In [53]:
df = real_df.append(fake_df)
df.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0


In [54]:
df.shape

(44898, 5)

Before we split the data, let's clean it a bit by removing duplicates.

In [55]:
df.drop_duplicates(subset='text', keep='last', inplace=True)
df.shape

(38646, 5)

In [56]:
df.describe()

,label
count,38646.000000
mean,0.451664
std,0.497665
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


To keep things simple, we will just connect the title and text together with just a space. If we'd want to apply this task to other texts, it would probably worth it to do a bit more preprocessing. For example, we may want to remove the `<news outlet>` that was added at the beginning of the titles or texts.

In [57]:
final_df = pd.DataFrame(df['label'])
final_df['text'] = pd.DataFrame(df['title'] + ' ' + df['text'])

In [58]:
final_df

,label,text
0,0,"As U.S. budget fight looms, Republicans flip t..."
1,0,U.S. military to accept transgender recruits o...
2,0,Senior U.S. Republican senator: 'Let Mr. Muell...
3,0,FBI Russia probe helped by Australian diplomat...
4,0,Trump wants Postal Service to charge 'much mor...
...,...,...
23476,1,McPain: John McCain Furious That Iran Treated ...
23477,1,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...
23478,1,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...
23479,1,How to Blow $700 Million: Al Jazeera America F...


## Splitting the Data

In [59]:
train, test = train_test_split(final_df, test_size=0.25, random_state=42)
test, val = train_test_split(test, test_size=0.5, random_state=42)

In [60]:
print(train.shape, val.shape, test.shape)

(28984, 2) (4831, 2) (4831, 2)


## Saving the Data

We can now save the data as a csv so that we can load it with `datasets`' `load_dataset` function.

In [61]:
train.to_csv('train.csv', index=False, header='text')
val.to_csv('val.csv', index=False, header='text')
test.to_csv('test.csv', index=False, header='text')

In [62]:
train_df = pd.read_csv('train.csv')
train_df.head()

,label,text
0,1,Trump Admits What He Would Have Done After Ca...
1,1,"Trump, Sanders Win Big In Michigan – Trump, Cl..."
2,0,Peru's president tells Trump he favors bridges...
3,0,Russia's Olympic ban strengthens Putin's re-el...
4,0,Factbox: Norway's close-fought election for pa...


We can now load the data as a DatasetDict via the `load_dataset` method.

In [63]:
data_files = {"train": f"train.csv", "val": f"val.csv", "test": f"test.csv"}
raw_datasets = load_dataset("csv", data_files=data_files)

Using custom data configuration default-909a846fa8946f45


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-909a846fa8946f45/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [64]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 28984
    })
    val: Dataset({
        features: ['label', 'text'],
        num_rows: 4831
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 4831
    })
})

In [65]:
raw_datasets['train'][0]

{'label': 1,
 'text': ' Trump Admits What He Would Have Done After Carly Fiorina Fell Off Stage (VIDEO) By now, you ve probably heard that at a recent Ted Cruz rally, Carly Fiorina fell off the stage right after announcing the candidate and his family. She s fine, don t worry.Yet, leave it to Donald Trump to use this moment to his advantage in front of his band of merry morons.Trump told his crowd in Carmel, Indiana: Carly s perfectly nice. By the way, she fell off the stage the other day. Did anybody see that? And Cruz didn t do anything. Even I would have helped her, OK? Basically saying,  even I, the biggest jerk on earth, would have given her a hand. Then he just kept going with it, because you know, Trump: They just showed it to me coming in. I said,  No, I didn t see it.  They just showed it to me and I said,  wow, that s really cruel.  She fell off   she just went down. She went a long way down, right? She went down right in front of him and he was talking. He kept talking. He d

This works like the `from_pretrained` method we saw for the models and tokenizers (except the cache directory is
_~/.cache/huggingface/dataset_ by default).

The `raw_datasets` object is a dictionary with three keys: `"train"`, `"test"` and `"unsupervised"`
(which correspond to the three splits of that dataset). We will use the `"train"` split for training and the
`"test"` split for validation.

To preprocess our data, we will need a tokenizer:

In [66]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

As we saw in [preprocessing](https://huggingface.co/docs/transformers/master/en/preprocessing), we can prepare the text inputs for the model with the following command (this is an
example, not a command you can execute):

In [67]:
# inputs = tokenizer(sentences, padding="max_length", truncation=True)

This will make all the samples have the maximum length the model can accept (here 512), either by padding or truncating
them.

However, we can instead apply these preprocessing steps to all the splits of our dataset at once by using the
`map` method:

In [68]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

We can now tokenize the dataset and remove the text column since it contains strings (the text) and cannot be computed like numbers can. We also rename the `label` column to `labels` since that is what is needed for the `transformers` model. Lastly, we make sure to set the format to torch so that the data works with the torch model on the backend of `transformers`. We could have used TensorFlow too, but you can look at the documentation if you'd like to use it.

In [69]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets = tokenized_datasets.with_format("torch")

  0%|          | 0/29 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

You can learn more about the map method or the other ways to preprocess the data in the 🤗 Datasets [documentation](https://huggingface.co/docs/datasets/).

Next we will generate a small subset of the training and validation set, to enable faster training:

In [70]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000)) 
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000)) 
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]

In [71]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'token_type_ids'],
        num_rows: 28984
    })
    val: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'token_type_ids'],
        num_rows: 4831
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'token_type_ids'],
        num_rows: 4831
    })
})

In [75]:
small_train_dataset

Dataset({
    features: ['attention_mask', 'input_ids', 'labels', 'token_type_ids'],
    num_rows: 1000
})

In all the examples below, we will always use `small_train_dataset` and `small_eval_dataset`. Just replace
them by their _full_ equivalent to train or evaluate on the full dataset.

<a id='trainer'></a>

If you look at our `tokenizer_function`, we do not use `padding="max_length"` here because we want to use dynamic padding instead of constant max padding. We want to use dynamic padding so that our model can train faster. Dynamic padding makes it so that the number of tokens for every batch is dependent on the example in the batch with the most tokens. Without dynamic padding, all batches will have the same size based on the max token length chosen at the beginning. This makes it so we are doing unnecessary operations during training, thereby increasing training time. The extra operations come from the unnecessary padding applied to a batch.

For example, let's say we have two sentences:

```
"I think this is an amazing workshop!"
and
"Huggingface is cool!"
```

But our max token length is 20, we will be applying an excess amount of padding in order to make both sentences the same size. That is the point of padding! To make all sentences match in number of tokens. For this batch, the max number of tokens is based on the first sentence, which is 8. Therefore, if we tokenize with dynamic padding, we will get:

```
"I think this is an amazing workshop !"
and
"Huggingface is cool ! <PAD> <PAD> <PAD> <PAD>"
```

Instead of (if max token length was uniformly 20 across batch):

```
"I think this is an amazing workshop ! <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>"
and
"Huggingface is cool ! <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>"
```

See how much unecessary calculations our model would need to do with the extra padding? That is why dynamic padding is useful.

That said, dynamic padding is optional and we only use it when training with GPUs or CPUs. We cannot use dynamic padding when training on a TPU.

We instantiate a `data_collator` object we will be using during tokenization:

In [73]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Now, let's just test a sample batch from the tokenized dataset. We specifically chose a batch where none of the examples have 512 tokens (which is the max for BERT).

In [74]:
samples = small_train_dataset[61:66]
samples = {k: v for k, v in samples.items() if k not in ["idx", "text"]}
[len(x) for x in samples["input_ids"]]

[441, 493, 512, 225, 512]

We can now check the batch to see the max token length. As you can see, the max number of tokens in the batch was 405 so the `data_collator` added padding to the examples with less than 271 so that each example in the batch would have the same token length and can run smoothly on GPU.

In [76]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

{'attention_mask': torch.Size([5, 512]),
 'input_ids': torch.Size([5, 512]),
 'labels': torch.Size([5]),
 'token_type_ids': torch.Size([5, 512])}

In [77]:
batch['input_ids'].shape

torch.Size([5, 512])

If we load the data using the Torch `DataLoader`, and then check the number of tokens for the first few batches, we have:

In [78]:
train_dataloader = DataLoader(tokenized_datasets["train"], batch_size=16, shuffle=True, collate_fn=data_collator)

for step, batch in enumerate(train_dataloader):
    print(batch["input_ids"].shape)
    if step > 5:
        break

torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])


As you can see, all of the first batches have at least one text example where the number of token was either equal to or more than 512 tokens. This means that the examples in each batch will get padding that gives them 512 tokens.

So, data collation will depend on the data you have. It may be that some datasets have much less tokens or much higher variance. Some datasets may not be affected as much by data collation as others.

## Fine-tuning in PyTorch with the Trainer API

In [79]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/nvBXf7s7vTI?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

Since PyTorch does not provide a training loop, the 🤗 Transformers library provides a [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer)
API that is optimized for 🤗 Transformers models, with a wide range of training options and with built-in features like
logging, gradient accumulation, and mixed precision.

First, let's define our model:

In [80]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

As you can see, we are using the `bert-base-cased` model like we did for the tokenizer (this is important because the text needs to be tokenized specifically for the model we are using!).

It's also worth pointing out the we add the `num_label=2` argument since the problem we are working on has two labels (positive or negative sentiment).

This will issue a warning about some of the pretrained weights not being used and some weights being randomly
initialized. That's because we are throwing away the pretraining head of the BERT model to replace it with a
classification head which is randomly initialized. We will fine-tune this model on our task, transferring the knowledge
of the pretrained model to it (which is why doing this is called transfer learning).

Then, to define our [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer), we will need to instantiate a
[TrainingArguments](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.TrainingArguments). This class contains all the hyperparameters we can tune for the
[Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) or the flags to activate the different training options it supports. Let's begin by
using all the defaults, the only thing we then have to provide is a directory in which the checkpoints will be saved:

In [81]:
training_args = TrainingArguments("test_trainer")

Then we can instantiate a [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) like this:

In [82]:
trainer = Trainer(
    model=model, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset, data_collator=data_collator
)

To fine-tune our model, we just need to call

In [83]:
trainer.train()

***** Running training *****
  Num examples = 1000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 375


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=375, training_loss=0.029260515848795573, metrics={'train_runtime': 172.3446, 'train_samples_per_second': 17.407, 'train_steps_per_second': 2.176, 'total_flos': 788860388402400.0, 'train_loss': 0.029260515848795573, 'epoch': 3.0})

which will start a training that you can follow with a progress bar, which should take a couple of minutes to complete
(as long as you have access to a GPU). It won't actually tell you anything useful about how well (or badly) your model
is performing however as by default, there is no evaluation during training, and we didn't tell the
[Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) to compute any metrics. Let's have a look on how to do that now!

To have the [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) compute and report metrics, we need to give it a `compute_metrics`
function that takes predictions and labels (grouped in a namedtuple called [EvalPrediction](https://huggingface.co/docs/transformers/master/en/internal/trainer_utils#transformers.EvalPrediction)) and
return a dictionary with string items (the metric names) and float values (the metric values).

The 🤗 Datasets library provides an easy way to get the common metrics used in NLP with the `load_metric` function.
here we simply use accuracy. Then we define the `compute_metrics` function that just convert logits to predictions
(remember that all 🤗 Transformers models return the logits) and feed them to `compute` method of this metric.

In [84]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

The compute function needs to receive a tuple (with logits and labels) and has to return a dictionary with string keys
(the name of the metric) and float values. It will be called at the end of each evaluation phase on the whole arrays of
predictions/labels.

To check if this works on practice, let's create a new [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) with our fine-tuned model:

In [86]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator
)
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


{'eval_accuracy': 1.0,
 'eval_loss': 3.876490882248618e-05,
 'eval_runtime': 19.7527,
 'eval_samples_per_second': 50.626,
 'eval_steps_per_second': 6.328}

which showed an accuracy of 100% in our case!

If you want to fine-tune your model and regularly report the evaluation metrics (for instance at the end of each
epoch), here is how you should define your training arguments:

In [87]:
training_args = TrainingArguments("test_trainer", evaluation_strategy="epoch")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


See the documentation of [TrainingArguments](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.TrainingArguments) for more options.


<a id='keras'></a>

<a id='pytorch_native'></a>

## Fine-tuning in native PyTorch

In [ ]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/Dh9CL8fyG80?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

You might need to restart your notebook at this stage to free some memory, or execute the following code:

In [ ]:
del model
del pytorch_model
del trainer
torch.cuda.empty_cache()

Let's now see how to achieve the same results as in [trainer section](#trainer) in PyTorch. First we need to
define the dataloaders, which we will use to iterate over batches. We just need to apply a bit of post-processing to
our `tokenized_datasets` before doing that to:

- remove the columns corresponding to values the model does not expect (here the `"text"` column)
- rename the column `"label"` to `"labels"` (because the model expect the argument to be named `labels`)
- set the format of the datasets so they return PyTorch Tensors instead of lists.

Our _tokenized_datasets_ has one method for each of those steps:

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

Now that this is done, we can easily define our dataloaders:

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

Next, we define our model:

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

We are almost ready to write our training loop, the only two things are missing are an optimizer and a learning rate
scheduler. The default optimizer used by the [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) is [AdamW](https://huggingface.co/docs/transformers/master/en/main_classes/optimizer_schedules#transformers.AdamW):

In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

Finally, the learning rate scheduler used by default is just a linear decay from the maximum value (5e-5 here) to 0:

In [ ]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

One last thing, we will want to use the GPU if we have access to one (otherwise training might take several hours
instead of a couple of minutes). To do this, we define a `device` we will put our model and our batches on.

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

We now are ready to train! To get some sense of when it will be finished, we add a progress bar over our number of
training steps, using the _tqdm_ library.

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

Note that if you are used to freezing the body of your pretrained model (like in computer vision) the above may seem a
bit strange, as we are directly fine-tuning the whole model without taking any precaution. It actually works better
this way for Transformers model (so this is not an oversight on our side). If you're not familiar with what "freezing
the body" of the model means, forget you read this paragraph.

Now to check the results, we need to write the evaluation loop. Like in the [trainer section](#trainer) we will
use a metric from the datasets library. Here we accumulate the predictions at each batch before computing the final
result when the loop is finished.

In [ ]:
metric= load_metric("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

With the tight interoperability between TensorFlow and PyTorch models, you can even save the model and then reload it
as a PyTorch model (or vice-versa):

In [ ]:
from transformers import AutoModelForSequenceClassification

model.save_pretrained("my_imdb_model")
tensorflow_model = AutoModelForSequenceClassification.from_pretrained("my_imdb_model", from_pt=True)

<a id='additional-resources'></a>

## Additional resources

To look at more fine-tuning examples you can refer to:

- [🤗 Transformers Examples](https://github.com/huggingface/transformers/tree/master/examples) which includes scripts
  to train on all common NLP tasks in PyTorch and TensorFlow.

- [🤗 Transformers Notebooks](https://huggingface.co/docs/transformers/master/en/notebooks) which contains various notebooks and in particular one per task (look for
  the _how to finetune a model on xxx_).